# Privacy-Preserving CNN

## Preparations

### modules

In [1]:
import numpy as np  # handling vectors and matrices
import matplotlib.pyplot as plt  # plotting
from keras.models import load_model  # reload pretrained tf models
from pnn_functions import (train_tf_model, pred_tf_model,
                           train_pt_model, pred_pt_model)  # plain neural net functions
from evaluation_functions import evaluate_forecasts  # evaluation functions
from preproc_functions import list_combine  # own functions
from nn_arguments import Arguments  # nn options
from neural_nets import TFUNET, TFMNET, PTUNET, PTMNET  # own defined neural nets as classes

from torchvision import models

# pysyft
import syft as sy
import torch  # pytorch
hook = sy.TorchHook(torch)  # hooking pysyft into pytorch
import torch.nn.functional as F
import torch.nn as nn

Using TensorFlow backend.


### options

In [2]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### datasets

#### univariate training

In [3]:
train_Xu = np.load(dir_data+'train_Xu.npy')
train_yu = np.load(dir_data+'train_yu.npy')

In [4]:
print(train_Xu.shape)
print(train_yu.shape)

(988, 14, 1)
(988, 7)


\> 980 Observations for X days for 1 Variable!

#### multivariate training

In [5]:
train_Xm = np.load(dir_data+'train_Xm.npy')
train_ym = np.load(dir_data+'train_ym.npy')

In [6]:
print(train_Xm.shape)
print(train_ym.shape)

(988, 14, 8)
(988, 7)


\> 980 Observations for X days for 8 Variables!

#### test set

In [7]:
test_Xu = np.load(dir_data+'test_Xs.npy')[:,:,:1]
test_Xm = np.load(dir_data+'test_Xs.npy')
test_y = np.load(dir_data+'test_ys.npy')

In [8]:
print(test_Xu.shape)
print(test_Xm.shape)
print(test_y.shape)

(59, 14, 1)
(59, 14, 8)
(59, 7)


## Models

### Options

In [9]:
args = Arguments()
# overwrite timesteps depending on data
args.n_timesteps = train_Xu.shape[1]

In [10]:
train_Xu.shape[1]

14

### load pretrained weights

#### tensorflow

In [ ]:
# create net objects
""""model_tfu = TFUNET(args.n_features_u, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)
model_tfm = TFMNET(args.n_features_m, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)""""""

In [ ]:
# load weights
""""model_tfu.load_weights(dir_data+'models/weights_tfu.h5')
model_tfm.load_weights(dir_data+'models/weights_tfm.h5')""""

#### pytorch

In [11]:
# create net objects
model_ptu = PTUNET(args.n_features_u, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)
model_ptm = PTMNET(args.n_features_m, args.n_filters, args.n_timesteps, 
                   args.n_linear, args.n_outputs)

In [12]:
# load weights
model_ptu.load_state_dict(torch.load(dir_data+'models/weights_ptu.pt'), strict=False)
model_ptm.load_state_dict(torch.load(dir_data+'models/weights_ptm.pt'), strict=False)

In [13]:
model_u = model_ptu
model_m = model_ptm
print(model_u)
print(model_m)

PTUNET(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
  (l1): Linear(in_features=96, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=7, bias=True)
)
PTMNET(
  (conv1): Conv1d(8, 32, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
  (l1): Linear(in_features=16, out_features=100, bias=True)
  (l2): Linear(in_features=100, out_features=7, bias=True)
)


## Distribute data and model between parties (all encrypted)

In [14]:
# different parties
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

SMPC works with integers so we need to convert floats to int via fix_precision.

In [15]:
# share test data
testXu = torch.tensor(test_Xu).fix_precision().share(alice, bob, charlie, crypto_provider=crypto_provider)
#testy = torch.tensor(test_y).fix_precision().share(alice, bob, charlie, crypto_provider=crypto_provider)

In [ ]:
#show dataset
bob._objects

In [ ]:
#show dataset
alice._objects

In [ ]:
#show dataset
charlie._objects

In [ ]:
#Dont do it! - Just for Testing
bobs_model = modelu.copy().send(bob)
alices_model = modelu.copy().send(alice)
charlie_model = modelu.copy().send(charlie)

In [16]:
# share model
modelptu = model_ptu.fix_precision().share(alice, bob,charlie, crypto_provider=crypto_provider)

In [ ]:
#not nessecary now
def test(model, X, y):
    model.eval()
    with torch.no_grad():
        pred = model(X)
        score, scores = evaluate_forecasts(y, pred)
    return score

In [27]:
def test(args, modelptu, testXu):
    modelptu.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testXu:
            output = modelptu(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testXu.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testXu.dataset),
        100. * correct / len(testXu.dataset)))

In [28]:
test(args, modelptu, testXu)

TypeError: can not serialize 'map' object

In [ ]:
test(args, model_ptu, private_test_loader)